In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import csv
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
import pandas as pd

# from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

[1760765164.080516] [bfd74565809f:5385 :f]        vfs_fuse.c:281  UCX  ERROR inotify_add_watch(/tmp) failed: No space left on device


In [2]:
!nvidia-smi

Sat Oct 18 05:26:06 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.5     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000000:17:00.0 Off |                    0 |
| N/A   48C    P0              80W / 300W |   8925MiB / 81920MiB |     73%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

In [4]:

# path = '/kaggle/input/marathi/mar_train.csv'
# path_val = '/kaggle/input/marathi/mar_valid.csv'
# path_test = '/kaggle/input/marathi/mar_test.csv'

lang = 'hin'
path = f"../../aks_dataset/{lang}/train.csv"
path_val = f"../../aks_dataset/{lang}/valid.csv"
path_test = f"../../aks_dataset/{lang}/test.csv"

In [5]:
df = pd.read_csv(path , header = None)
df_val = pd.read_csv(path_val , header = None)
df_test = pd.read_csv(path_test , header = None)
english_words_val = df_val[0]
marathi_words_val = df_val[1]
english_words_test = df_test[0]
marathi_words_test = df_test[1]
english_words = df[0]
marathi_words = df[1]

In [6]:
# creating list of charecters in both languages

english_char_list = []
max_length_word_english = -1
for word in english_words:
  max_length_word_english = max(max_length_word_english,len(word)) 
  for char in word :
    english_char_list.append(char);
english_char_list = list(set(english_char_list))
english_char_list.sort()

marathi_char_list = []
max_length_word_marathi = -1
for word in marathi_words:
  max_length_word_marathi = max(max_length_word_marathi,len(word))
  for char in word :
    marathi_char_list.append(char);
marathi_char_list = list(set(marathi_char_list))
marathi_char_list.sort()


# finding out the maximum size word for english and marathi from validation and test data.
for word in english_words_val:
  max_length_word_english = max(max_length_word_english,len(word))
for word in english_words_test:
  max_length_word_english = max(max_length_word_english,len(word)) 
for word in marathi_words_val:
  max_length_word_marathi = max(max_length_word_marathi,len(word))
for word in marathi_words_test:
  max_length_word_marathi = max(max_length_word_marathi,len(word))


In [7]:
# english word to vector size = 27 ie. max_length_word_english
# marathi word to vector size = 20 ie. max_length_word_marathi
# for one word.
def word2vec(word, lang):
  vec = []
  if(lang == "english"):
    vec.append(len(english_char_list) + 1)
    for char in word:
      for i in range(len(english_char_list)):
        if(english_char_list[i] == char):
          vec.append(i+1)
    while(len(vec) < max_length_word_english + 1): # padding with max_length + 1.
        vec.append(0)
    vec.append(0)
  else :
    vec.append(len(marathi_char_list) + 1)
    for char in word:
      for i in range(len(marathi_char_list)):
        if( marathi_char_list[i] == char):
          vec.append(i+1)
    while(len(vec) < max_length_word_marathi + 1):  # padding with max_length + 1.
        vec.append(0)
    vec.append(0)
  return(vec)


In [8]:
vec = word2vec(marathi_words[10],"marathi")
print(marathi_words[10])
print(vec)

ज़ोला
[65, 23, 50, 62, 43, 51, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [9]:
# creating matrix of representation for whole words of english and marathi.

def ip_matrix_construct(words, lang):
  ans = []
  for word in words:
    ans.append(word2vec(word, lang))
  return(ans)

In [10]:
# calculated representations of whole english and marathi words in variables english and marathi matrix.
english_matrix = ip_matrix_construct(english_words, "english")
marathi_matrix = ip_matrix_construct(marathi_words, "marathi")
english_matrix = torch.tensor(english_matrix)
marathi_matrix = torch.tensor(marathi_matrix)

english_matrix_val = ip_matrix_construct(english_words_val, "english")
marathi_matrix_val = ip_matrix_construct(marathi_words_val, "marathi")
english_matrix_val = torch.tensor(english_matrix_val)
marathi_matrix_val = torch.tensor(marathi_matrix_val)
english_matrix_test = ip_matrix_construct(english_words_test, "english")
marathi_matrix_test =ip_matrix_construct(marathi_words_test, "marathi")
english_matrix_test = torch.tensor(english_matrix_test)
marathi_matrix_test = torch.tensor(marathi_matrix_test)

In [11]:
class Encoder(nn.Module):
  def __init__(self,input_size, embedding_size, hidden_size, enc_layers, p, cell_type, bidirectional):
    super(Encoder,self).__init__()
    self.hidden_size = hidden_size
    self.enc_layers = enc_layers
    self.dropout = nn.Dropout(p)
    self.cell_type = cell_type
    self.bidirectional = bidirectional
    self.embedding = nn.Embedding(input_size, embedding_size)
    if(cell_type == "GRU"):
      self.gru = nn.GRU(embedding_size, hidden_size, enc_layers, dropout = p, bidirectional = bidirectional)
    if(cell_type == "RNN"):
      self.rnn = nn.RNN(embedding_size, hidden_size, enc_layers, dropout = p, bidirectional = bidirectional)
    if(cell_type == "LSTM"):
      self.lstm = nn.LSTM(embedding_size, hidden_size, enc_layers, dropout = p, bidirectional = bidirectional)

  def forward(self, x):
    embedding = self.dropout(self.embedding(x))
    if(self.cell_type == "GRU"):
      output, hidden = self.gru(embedding)
    if(self.cell_type == "RNN"):
      output, hidden = self.rnn(embedding)
    if(self.cell_type == "LSTM"):
      outputs, (hidden,cell) = self.lstm(embedding)
      return outputs, hidden, cell
    return output, hidden

  def initHidden(self):
    return torch.zeros(1, 1, self.hidden_size, device=device)

In [12]:
class Decoder(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, output_size, dec_layers, p, cell_type):
    super(Decoder, self).__init__()
    self.hidden_size = hidden_size
    self.dec_layers = dec_layers
    self.dropout = nn.Dropout(p)
    self.cell_type = cell_type
    self.embedding = nn.Embedding(input_size, embedding_size)
    if(cell_type == "GRU"):
      self.gru = nn.GRU(embedding_size, hidden_size, dec_layers, dropout = p)
    if(cell_type == "RNN"):
      self.rnn = nn.RNN(embedding_size, hidden_size, dec_layers, dropout = p)
    if(cell_type == "LSTM"):
      self.lstm = nn.LSTM(embedding_size, hidden_size, dec_layers, dropout = p)
    self.fc = nn.Linear(hidden_size, output_size)  # fully connected.
  
  def forward(self,x,output, hidden, cell = 0):
    x = x.unsqueeze(0).int()
    embedding = self.dropout(self.embedding(x))
    if(self.cell_type == "GRU"):
        outputs, hidden = self.gru(embedding, hidden)
    if(self.cell_type == "RNN"):
        outputs, hidden = self.rnn(embedding, hidden)
    if(self.cell_type == "LSTM"):
        outputs, (hidden, cell) = self.lstm(embedding, (hidden, cell))
    # shape of outputs: (1, N, hidden_size)
    predictions = self.fc(outputs)
    # shape of predictions: (1, N, length_of_vocab)
    predictions = predictions.squeeze(0)
    # shape of predictions: (N, length_of_vocab)
    if(self.cell_type == "LSTM"):
        return predictions, hidden, cell
    return predictions, hidden


  def initHidden(self):
    return torch.zeros(1, 1, self.hidden_size, device=device)

In [13]:
class Atten_decoder(nn.Module):
  def __init__(self, input_size, embedding_size, hidden_size, output_size, dec_layers, p, cell_type, bidirectional):
    super(Atten_decoder, self).__init__()
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.max_length = len(english_matrix[0])  
    self.dec_layers = dec_layers
    self.dropout = nn.Dropout(p)
    self.cell_type = cell_type
    self.embedding = nn.Embedding(input_size, embedding_size)
    if(cell_type == "GRU"):
      self.gru = nn.GRU(hidden_size, hidden_size, dec_layers, dropout = p)
    if(cell_type == "RNN"):
      self.rnn = nn.RNN(hidden_size, hidden_size, dec_layers, dropout = p)
    if(cell_type == "LSTM"):
      self.lstm = nn.LSTM(hidden_size, hidden_size, dec_layers, dropout = p)
    self.fc = nn.Linear(hidden_size, output_size)  # fully connected.
    self.attn = nn.Linear(hidden_size+embedding_size, self.max_length)
    if(bidirectional):
      self.attn_combine = nn.Linear(hidden_size * 2 + embedding_size, hidden_size)
    else :
      self.attn_combine = nn.Linear(hidden_size + embedding_size, hidden_size)

  def forward(self, x,output, hidden, cell = 0):
    x = x.unsqueeze(0)
    output=output.permute(1,0,2)
    embedded = self.embedding(x)
    embedded = self.dropout(embedded)
    attn_weights = F.softmax(self.attn(torch.cat((embedded[0],hidden[0]), 1)), dim = 1)
    attn_applied = torch.bmm(attn_weights.unsqueeze(1),output)
    attn_applied = attn_applied.squeeze(1)
    op = torch.cat((embedded[0], attn_applied), 1)

    op = self.attn_combine(op).unsqueeze(0)
    op = F.relu(op)
    if(self.cell_type == "GRU"):
        outputs, hidden = self.gru(op, hidden)
    if(self.cell_type == "RNN"):
        outputs, hidden = self.rnn(op, hidden)
    if(self.cell_type == "LSTM"):
        outputs, (hidden, cell) = self.lstm(op, (hidden, cell))
    predictions = self.fc(outputs)
    predictions = predictions.squeeze(0)
    if(self.cell_type == "LSTM"):
        return predictions, hidden, cell
    return predictions, hidden



In [14]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, cell_type, bidirectional, enc_layers, dec_layers):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.cell_type = cell_type
        self.bidirectional = bidirectional
        self.enc_layers = enc_layers
        self.dec_layers = dec_layers

    def forward(self, source, target, teacher_force_ratio=0.5):
        batch_size = source.shape[1]
        target_len = target.shape[0]
        target_vocab_size = len(marathi_char_list) + 2  
        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)
        if(self.cell_type == "LSTM"):
            encoder_output, hidden, cell = self.encoder(source)
        else:
            encoder_output, hidden = self.encoder(source)
        if(self.enc_layers != self.dec_layers or self.bidirectional == True):
          hidden = hidden[self.enc_layers - 1] + hidden[self.enc_layers - 1]
          hidden = hidden.repeat(self.dec_layers,1,1)
          if(self.cell_type == "LSTM"):
              cell = cell[self.enc_layers - 1] + cell[self.enc_layers - 1]
              cell = cell.repeat(self.dec_layers,1,1)
        
        x = target[0]
    
        for t in range(1, target_len):
            if(self.cell_type == "LSTM"):
                output, hidden, cell = self.decoder(x, encoder_output, hidden, cell)
            else :
                output, hidden = self.decoder(x, encoder_output, hidden)
            outputs[t] = output

            best_guess = output.argmax(1)

            x = target[t] if random.random() < teacher_force_ratio else best_guess
            
        return outputs


In [15]:
def Accuracy(model, english_matrix, marathi_matrix, epoch, batch_size):
    correct_count = 0
    for batch_idx in range((int)(len(english_matrix) / batch_size)):
        inp_data = english_matrix[batch_size * batch_idx : batch_size * (batch_idx+1)].to(device)
        target = marathi_matrix[batch_size * batch_idx : batch_size * (batch_idx+1)].to(device)
        output = model.forward(inp_data.T, target.T, 0)
        output = nn.Softmax(dim=2)(output)
        output = torch.argmax(output, dim=2)
        output = output.T
        for i in range(batch_size):
            if(torch.equal(output[i][1:],target[i][1:])):
                correct_count += 1
    accuracy = correct_count * 100 / len(english_matrix)
    return accuracy
        

In [16]:
# creating list of expected marathi word and predicted marathi word.
predictions_vanilla_train = []
predictions_vanilla_val = []
predictions_vanilla_test = []
def matrix_to_words(model, english_matrix, marathi_matrix, batch_size, data_type):
  for batch_idx in range((int)(len(english_matrix) / batch_size)+1):
        inp_data = english_matrix[batch_size * batch_idx : batch_size * (batch_idx+1)].to(device)
        target = marathi_matrix[batch_size * batch_idx : batch_size * (batch_idx+1)].to(device)
        output = model.forward(inp_data.T, target.T, 0)
        output = nn.Softmax(dim=2)(output)
        output = torch.argmax(output, dim=2)
        output = output.T
        for i in range(len(target)):
          target_word = target[i]
          output_word = output[i]
          word1 = ""
          word2 = ""
          for j in range(len(target_word)):
            if(target_word[j]>0 and target_word[j]<64):
              word1 += marathi_char_list[target_word[j] - 1]
          for j in range(len(output_word)):
            if(output_word[j]>0 and output_word[j]<64):
              word2 += marathi_char_list[output_word[j] - 1]
          temp = [word1, word2]
          if(data_type == "train"):
            predictions_vanilla_train.append(temp)
          if(data_type == "validation"):
            predictions_vanilla_val.append(temp)
          if(data_type == "test"):
            predictions_vanilla_test.append(temp)

In [17]:
from tqdm import tqdm
def neural_network(cell_type, bidirectional, enc_layers, dec_layers, batch_size, embedding_size, hidden_size, enc_dropout, dec_dropout, attention):
    learning_rate = 1e-3
    num_epochs = 20
    input_size_encoder = len(english_char_list) + 2  
    input_size_decoder = len(marathi_char_list) + 2  
    output_size        = len(marathi_char_list) + 2  

    encoder_net = Encoder(input_size_encoder, embedding_size, hidden_size, enc_layers, enc_dropout, cell_type,bidirectional).to(device)
    if(attention):
        decoder_net = Atten_decoder(input_size_decoder,embedding_size,hidden_size,output_size,dec_layers,dec_dropout, cell_type, bidirectional).to(device)
    else:
        decoder_net = Decoder(input_size_decoder,embedding_size,hidden_size,output_size,dec_layers,dec_dropout, cell_type).to(device)

    model = Seq2Seq(encoder_net, decoder_net, cell_type, bidirectional, enc_layers, dec_layers).to(device)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    pad_idx = len(marathi_char_list) + 1  # 64 # pading index for marathi
    criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)


    for epoch in tqdm(range(num_epochs)):
        print("epoch = ",epoch)

        model.train()
        total_loss = 0
        step = 0
        for batch_idx in range((int)(len(english_matrix) / batch_size)):
            inp_data = english_matrix[batch_size * batch_idx : batch_size * (batch_idx+1)].to(device)
            target = marathi_matrix[batch_size * batch_idx : batch_size * (batch_idx+1)].to(device)
            target = target.T
            output = model(inp_data.T, target)

            output = output[1:].reshape(-1, output.shape[2])
            target = target[1:].reshape(-1)
            optimizer.zero_grad()
            loss = criterion(output, target)
            total_loss += loss
            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

            optimizer.step()

            step += 1
        print("total loss = ",total_loss / step)
        training_accuracy = Accuracy(model, english_matrix, marathi_matrix, epoch, batch_size)
        print("Training Accuracy = ", training_accuracy)
        val_accuracy = Accuracy(model, english_matrix_val, marathi_matrix_val, epoch, batch_size)
        print("Validation accuracy = ",val_accuracy)
    return model

In [18]:
cell_type = "LSTM"
bidirectional = True
enc_layers = 2
dec_layers = 2
batch_size = 256
embedding_size = 256
hidden_size = 512
enc_dropout = 0
dec_dropout = 0
attention = False


model = neural_network(cell_type, bidirectional, enc_layers, dec_layers, batch_size, embedding_size, hidden_size, enc_dropout, dec_dropout, attention)

  0%|                                                                                            | 0/20 [00:00<?, ?it/s]

epoch =  0
total loss =  tensor(0.5212, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  20.792


  5%|████▏                                                                               | 1/20 [01:00<19:03, 60.19s/it]

Validation accuracy =  24.335378323108383
epoch =  1
total loss =  tensor(0.2561, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  30.609


 10%|████████▍                                                                           | 2/20 [01:40<14:37, 48.77s/it]

...

Validation accuracy =  38.194116721724086
epoch =  18
total loss =  tensor(0.0350, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  80.156


 95%|██████████████████████████████████████████████████████████████████████████████▊    | 19/20 [14:04<00:44, 44.44s/it]

Validation accuracy =  38.93345917885795
epoch =  19
total loss =  tensor(0.0337, device='cuda:0', grad_fn=<DivBackward0>)
Training Accuracy =  80.383


100%|███████████████████████████████████████████████████████████████████████████████████| 20/20 [14:49<00:00, 44.46s/it]

Validation accuracy =  37.879502910177756


In [18]:
# cell_type = "LSTM"
# bidirectional = False
# enc_layers = 2
# dec_layers = 2
# batch_size = 256
# embedding_size = 256
# hidden_size = 512
# enc_dropout = 0
# dec_dropout = 0
# attention = True

In [19]:
model_name = f"models/model_{cell_type}_{'bi' if bidirectional else 'uni'}_{'attn' if attention else 'noattn'}.pt"
print(model_name)

models/model_LSTM_uni_attn.pt


In [20]:
# Save the entire model
torch.save(model, model_name)
print("Model saved successfully!")

Model saved successfully!


In [20]:
model = torch.load(model_name)
model.eval()  # Important: set to eval mode before testing

Seq2Seq(
  (encoder): Encoder(
    (dropout): Dropout(p=0, inplace=False)
    (embedding): Embedding(28, 256)
    (lstm): LSTM(256, 512, num_layers=2)
  )
  (decoder): Atten_decoder(
    (dropout): Dropout(p=0, inplace=False)
    (embedding): Embedding(66, 256)
    (lstm): LSTM(512, 512, num_layers=2)
    (fc): Linear(in_features=512, out_features=66, bias=True)
    (attn): Linear(in_features=768, out_features=31, bias=True)
    (attn_combine): Linear(in_features=768, out_features=512, bias=True)
  )
)

In [21]:
matrix_to_words(model, english_matrix_test, marathi_matrix_test, batch_size, "test")

In [22]:
import csv

# --- define data ---
filename_test = 'predictions/uni_attn.csv'

# english_words: list of English strings
# predictions_vanilla_test: list of [actual_word, predicted_word] pairs



# Combine into rows: [English, Actual, Predicted]
rows = []
for eng, (actual, predicted) in zip(english_words_test, predictions_vanilla_test):
    rows.append([eng, actual, predicted])

# --- write to CSV ---
with open(filename_test, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['eng', 'actual', 'predicted'])  # header
    writer.writerows(rows)

print(f'✅ CSV file \"{filename_test}\" has been created successfully.')

✅ CSV file "predictions/uni_attn.csv" has been created successfully.


In [27]:
import csv

# --- define data ---
filename_test = 'predictions/bi_no_attn.csv'

# english_words: list of English strings
# predictions_vanilla_test: list of [actual_word, predicted_word] pairs



# Combine into rows: [English, Actual, Predicted]
rows = []
for eng, (actual, predicted) in zip(english_words_test, predictions_vanilla_test):
    rows.append([eng, actual, predicted])

# --- write to CSV ---
with open(filename_test, 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['eng', 'actual', 'predicted'])  # header
    writer.writerows(rows)

print(f'✅ CSV file \"{filename_test}\" has been created successfully.')

✅ CSV file "predictions/bi_no_attn.csv" has been created successfully.


In [24]:
# creating csv file of predictions
filename_test = 'bi_no_attn_predictions_vanilla_test.csv'

with open(filename_test, 'w', newline='') as file:
    writer = csv.writer(file, quoting=csv.QUOTE_NONE)
    writer.writerows(predictions_vanilla_test)

print(f'CSV file "{filename_test}" has been created.')

CSV file "predictions_vanilla_test.csv" has been created.


In [17]:
import pandas as pd

def calculate_accuracies(csv_path):
    # Read CSV (no headers)
    df = pd.read_csv(csv_path, header=None, sep=',', names=['actual', 'predicted'])
    
    # Drop any empty rows
    df = df.dropna()
    
    total_words = len(df)
    correct_words = 0
    total_chars = 0
    correct_chars = 0

    for _, row in df.iterrows():
        actual = str(row['actual']).strip()
        predicted = str(row['predicted']).strip()

        # --- Word-level accuracy ---
        if actual == predicted:
            correct_words += 1

        # --- Character-level accuracy ---
        max_len = max(len(actual), len(predicted))
        total_chars += max_len
        for a, b in zip(actual, predicted):
            if a == b:
                correct_chars += 1

    word_acc = correct_words / total_words * 100 if total_words > 0 else 0
    char_acc = correct_chars / total_chars * 100 if total_chars > 0 else 0

    print(f"✅ Word-level accuracy: {word_acc:.2f}%")
    print(f"✅ Character-level accuracy: {char_acc:.2f}%")

    return word_acc, char_acc

In [19]:
csv_path = "predictions_vanilla_test.csv"
calculate_accuracies(csv_path)

✅ Word-level accuracy: 39.45%
✅ Character-level accuracy: 71.56%


(39.453125, 71.55517656195276)

In [21]:
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name = 'models/model_LSTM_bi_noattn.pt'
# === 1. Load saved model ===
model = torch.load(model_name, map_location=device)
model.eval()
print("✅ Model loaded successfully")

# === 2. Helper function to convert English word to tensor ===
def encode_word(word, lang_dict, max_len):
    tensor = torch.zeros(max_len, dtype=torch.long)
    for i, ch in enumerate(word):
        if ch in lang_dict:
            tensor[i] = lang_dict[ch] + 1
        else:
            tensor[i] = 0  # unknown char or padding
    return tensor.unsqueeze(1).to(device)  # shape: (seq_len, 1)

# === 3. Helper function to decode tensor back to Marathi string ===
def decode_tensor(tensor, marathi_char_list):
    word = ""
    for idx in tensor:
        if 0 < idx < len(marathi_char_list) + 1:
            word += marathi_char_list[idx - 1]
    return word

# === 2. Predict for a single English word ===
def predict_word(model, english_word):
    # convert to numeric tensor using your own function
    eng_vec = word2vec(english_word, "english")
    inp_tensor = torch.tensor(eng_vec, dtype=torch.long).unsqueeze(1).to(device)  # shape [seq_len, 1]
    
    # dummy target tensor (zeroed out)
    target_tensor = torch.zeros(max_length_word_marathi + 2, 1, dtype=torch.long).to(device)
    
    with torch.no_grad():
        output = model.forward(inp_tensor, target_tensor, 0)  # teacher forcing = 0
        output = nn.Softmax(dim=2)(output)
        output = torch.argmax(output, dim=2).squeeze(1)  # shape: [seq_len]
    
    # decode the predicted indices into Marathi characters
    predicted_word = ""
    for idx in output:
        idx = idx.item()
        if 0 < idx <= len(marathi_char_list):
            predicted_word += marathi_char_list[idx - 1]
    
    return predicted_word

✅ Model loaded successfully


In [33]:
# === 5. Example usage ===
english_word = "kaustubh"  # any test English transliteration
predicted_marathi = predict_word(
    model, 
    english_word
)
print(f"English: {english_word}")
print(f"Predicted Marathi: {predicted_marathi}")

English: kaustubh
Predicted Marathi: कुस्तूभ


In [22]:
import pandas as pd
import torch
from tqdm import tqdm

# --- Load the test CSV (no headers) ---
original_csv = "../../aks_dataset/hin/test.csv"
df_test = pd.read_csv(original_csv, header=None)

# --- Give column names to df_test ---
df_test.columns = ['eng', 'actual']  # first column = English, second column = reference Hindi

# --- Setup device ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Make sure your model is on the device
model.to(device)
model.eval()  # Set to evaluation mode

# --- Predict function wrapper ---
def predict_words_batch(model, words):
    """Predict multiple words using GPU."""
    predictions = []
    with torch.no_grad():
        for word in tqdm(words):
            word = str(word).strip()
            pred = predict_word(model, word)  # your existing function
            predictions.append(pred)
    return predictions

# --- Get first column (English words) ---
first_column = df_test['eng'].tolist()

# --- Run predictions ---
predicted = predict_words_batch(model, first_column)

# --- Save predictions back to DataFrame ---
df_test['predicted'] = predicted

# --- Save to CSV ---
df_test.to_csv("bi_no_attn_lstm_predictions.csv", index=False)
print("Predictions saved to bi_lstm_predictions.csv")

Using device: cuda


100%|█████████████████████████████████████████████████████████████████████████████| 10112/10112 [02:08<00:00, 78.42it/s]


Predictions saved to bi_lstm_predictions.csv
